In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import lietorch
import torch
from gecco_torch.additional_metrics.metrics_so3 import geodesic_distance, rotational_distance_between_pairs_dot_product
from gecco_torch.utils.riemannian_helper_functions import exp_map_at_L, log_map_at_L, find_cholesky_L, group_operation_add, group_operation_inverse_L
from gecco_torch.utils.build_cov_matrix_torch import build_covariance_from_scaling_rotation_xyzw
from gecco_torch.diffusionsplat import LogUniformSchedule

/home/giese/miniconda3/envs/gecco/lib/python3.10/site-packages/jaxtyping/__init__.py:243: UserWarning: jaxtyping version >=0.2.23 should be used with Equinox version >=0.11.1
  warnings.warn(


## Wie kann man Schritte von einer Rotation zur anderen machen?

In [3]:
rotation_ziel = lietorch.SO3([], from_uniform_sampled=1).cuda()
start_rotation = lietorch.SO3(torch.tensor([[0.0, 0.0, 0.0, 1.0]])).cuda() # unit rotation
schedule = LogUniformSchedule(165).return_schedule(128)
reverse_schedule = torch.flip(schedule, [0])

### In einem Schritt

In [4]:
print(f"Am anfang ist die rotation distanz zwischen den beiden Rotationen: {rotational_distance_between_pairs_dot_product(start_rotation.vec(), rotation_ziel.vec())}")
# Compute the relative rotation from R1 to R2
relative_rotation = rotation_ziel * start_rotation.inv()
relative_rotation_log = relative_rotation.log() #/ t_cur
step_size = 1
small_step_rotvec = step_size * relative_rotation_log
small_step_rotation = lietorch.SO3.exp(small_step_rotvec)
new_rotation = small_step_rotation * start_rotation
print(f"Am Ende ist die rotation distanz zwischen den beiden Rotationen: {rotational_distance_between_pairs_dot_product(new_rotation.vec(), rotation_ziel.vec())}")
print(new_rotation.vec(), rotation_ziel.vec())

Am anfang ist die rotation distanz zwischen den beiden Rotationen: tensor([2.2835], device='cuda:0')
Am Ende ist die rotation distanz zwischen den beiden Rotationen: tensor([0.], device='cuda:0')
tensor([[ 0.0101, -0.8073, -0.4185,  0.4160]], device='cuda:0') tensor([[-0.0101,  0.8073,  0.4185, -0.4160]], device='cuda:0')


In [5]:
ts = list(enumerate(zip(reverse_schedule[:-1], reverse_schedule[1:])))

#### First order

In [25]:
print(f"Am anfang ist die rotation distanz zwischen den beiden Rotationen: {rotational_distance_between_pairs_dot_product(start_rotation.vec(), rotation_ziel.vec())}")
rotation_next = start_rotation
for i, (t_cur, t_next) in ts:
    # mache einen Schritt von start rotation in Richtung ziel rotation
    step_size = t_next - t_cur
    # Compute the relative rotation from R1 to R2
    relative_rotation = rotation_ziel * rotation_next.inv()
    relative_rotation_log = relative_rotation.log() / t_cur
    small_step_rotvec = - step_size * relative_rotation_log
    small_step_rotation = lietorch.SO3.exp(small_step_rotvec)
    new_rotation = small_step_rotation * rotation_next
    rotation_next = new_rotation
    print(f"Nach {i} Schritten ist die rotation distanz zwischen den beiden Rotationen: {rotational_distance_between_pairs_dot_product(new_rotation.vec(), rotation_ziel.vec())}")
print(new_rotation.vec(), rotation_ziel.vec())

Am anfang ist die rotation distanz zwischen den beiden Rotationen: tensor([2.2835], device='cuda:0')
Nach 0 Schritten ist die rotation distanz zwischen den beiden Rotationen: tensor([2.0887], device='cuda:0')
Nach 1 Schritten ist die rotation distanz zwischen den beiden Rotationen: tensor([1.9106], device='cuda:0')
Nach 2 Schritten ist die rotation distanz zwischen den beiden Rotationen: tensor([1.7477], device='cuda:0')
Nach 3 Schritten ist die rotation distanz zwischen den beiden Rotationen: tensor([1.5986], device='cuda:0')
Nach 4 Schritten ist die rotation distanz zwischen den beiden Rotationen: tensor([1.4623], device='cuda:0')
Nach 5 Schritten ist die rotation distanz zwischen den beiden Rotationen: tensor([1.3376], device='cuda:0')
Nach 6 Schritten ist die rotation distanz zwischen den beiden Rotationen: tensor([1.2235], device='cuda:0')
Nach 7 Schritten ist die rotation distanz zwischen den beiden Rotationen: tensor([1.1192], device='cuda:0')
Nach 8 Schritten ist die rotation d

#### Second order

In [31]:
print(f"Am anfang ist die rotation distanz zwischen den beiden Rotationen: {rotational_distance_between_pairs_dot_product(start_rotation.vec(), rotation_ziel.vec())}")
rotation_next = start_rotation
for i, (t_cur, t_next) in ts:
    # mache einen Schritt von start rotation in Richtung ziel rotation
    step_size = t_next - t_cur
    # Compute the relative rotation from R1 to R2
    relative_rotation = rotation_ziel.inv() * rotation_next
    relative_rotation_log = relative_rotation.log() / t_cur
    small_step_rotvec = step_size * relative_rotation_log
    small_step_rotation = lietorch.SO3.exp(small_step_rotvec)
    new_rotation = small_step_rotation * rotation_next
    rotation_next = new_rotation
    print(f"Nach {i} Schritten first: ist die rotation distanz zwischen den beiden Rotationen: {rotational_distance_between_pairs_dot_product(new_rotation.vec(), rotation_ziel.vec())}")
    if i < len(ts) - 1:
        relative_rotation = rotation_ziel.inv() * rotation_next
        relative_rotation_log_2nd = relative_rotation.log() / t_next
        direction = relative_rotation_log_2nd / 2 + relative_rotation_log / 2
        small_step_rotation = lietorch.SO3.exp(step_size * direction)
        new_rotation = small_step_rotation * rotation_next
        rotation_next = new_rotation
        print(f"Nach {i} Schritten second: ist die rotation distanz zwischen den beiden Rotationen: {rotational_distance_between_pairs_dot_product(new_rotation.vec(), rotation_ziel.vec())}")
print(new_rotation.vec(), rotation_ziel.vec())

Am anfang ist die rotation distanz zwischen den beiden Rotationen: tensor([2.2835], device='cuda:0')
Nach 0 Schritten first: ist die rotation distanz zwischen den beiden Rotationen: tensor([2.0887], device='cuda:0')
Nach 0 Schritten second: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.8940], device='cuda:0')
Nach 1 Schritten first: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.7325], device='cuda:0')
Nach 1 Schritten second: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.5710], device='cuda:0')
Nach 2 Schritten first: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.4370], device='cuda:0')
Nach 2 Schritten second: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.3030], device='cuda:0')
Nach 3 Schritten first: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.1919], device='cuda:0')
Nach 3 Schritten second: ist die rotation distanz zwischen den beiden Rotationen: tensor([1

In [8]:
print(f"Am anfang ist die rotation distanz zwischen den beiden Rotationen: {rotational_distance_between_pairs_dot_product(start_rotation.vec(), rotation_ziel.vec())}")
rotation_next = start_rotation
for i, (t_cur, t_next) in ts:
    # mache einen Schritt von start rotation in Richtung ziel rotation
    step_size = t_next - t_cur
    # Compute the relative rotation from R1 to R2
    relative_rotation = rotation_ziel * rotation_next.inv()
    relative_rotation_log = relative_rotation.log() / t_cur
    small_step_rotvec = - step_size * relative_rotation_log
    small_step_rotation = lietorch.SO3.exp(small_step_rotvec)
    new_rotation = small_step_rotation * rotation_next
    rotation_next = new_rotation
    print(f"Nach {i} Schritten first: ist die rotation distanz zwischen den beiden Rotationen: {rotational_distance_between_pairs_dot_product(new_rotation.vec(), rotation_ziel.vec())}")
    if i < len(ts) - 1:
        relative_rotation = rotation_ziel * rotation_next.inv()
        relative_rotation_log_2nd = relative_rotation.log() / t_next
        direction = relative_rotation_log_2nd / 2 + relative_rotation_log / 2
        small_step_rotation = lietorch.SO3.exp(- step_size * direction)
        new_rotation = small_step_rotation * rotation_next
        rotation_next = new_rotation
        print(f"Nach {i} Schritten second: ist die rotation distanz zwischen den beiden Rotationen: {rotational_distance_between_pairs_dot_product(new_rotation.vec(), rotation_ziel.vec())}")
print(new_rotation.vec(), rotation_ziel.vec())

Am anfang ist die rotation distanz zwischen den beiden Rotationen: tensor([2.2835], device='cuda:0')
Nach 0 Schritten first: ist die rotation distanz zwischen den beiden Rotationen: tensor([2.0887], device='cuda:0')
Nach 0 Schritten second: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.8940], device='cuda:0')
Nach 1 Schritten first: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.7325], device='cuda:0')
Nach 1 Schritten second: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.5710], device='cuda:0')
Nach 2 Schritten first: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.4370], device='cuda:0')
Nach 2 Schritten second: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.3030], device='cuda:0')
Nach 3 Schritten first: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.1919], device='cuda:0')
Nach 3 Schritten second: ist die rotation distanz zwischen den beiden Rotationen: tensor([1

In [9]:
print(f"Am anfang ist die rotation distanz zwischen den beiden Rotationen: {rotational_distance_between_pairs_dot_product(start_rotation.vec(), rotation_ziel.vec())}")
rotation_next = start_rotation
for i, (t_cur, t_next) in ts:
    # mache einen Schritt von start rotation in Richtung ziel rotation
    step_size = t_next - t_cur
    # Compute the relative rotation from R1 to R2
    relative_rotation = rotation_ziel * rotation_next.inv()
    relative_rotation_log = relative_rotation.log() / t_cur
    small_step_rotvec = - step_size * relative_rotation_log
    small_step_rotation = lietorch.SO3.exp(small_step_rotvec)
    new_rotation = small_step_rotation * rotation_next
    rotation_next = new_rotation
    print(f"Nach {i} Schritten first: ist die rotation distanz zwischen den beiden Rotationen: {rotational_distance_between_pairs_dot_product(new_rotation.vec(), rotation_ziel.vec())}")
    if i < len(ts) - 1:
        relative_rotation = rotation_ziel * rotation_next.inv()
        relative_rotation_log_2nd = relative_rotation.log() / t_next
        direction = relative_rotation_log_2nd / 2 + relative_rotation_log / 2
        small_step_rotation = lietorch.SO3.exp(- step_size * direction)
        new_rotation = small_step_rotation * rotation_next
        rotation_next = new_rotation
        print(f"Nach {i} Schritten second: ist die rotation distanz zwischen den beiden Rotationen: {rotational_distance_between_pairs_dot_product(new_rotation.vec(), rotation_ziel.vec())}")
print(new_rotation.vec(), rotation_ziel.vec())

Am anfang ist die rotation distanz zwischen den beiden Rotationen: tensor([2.2835], device='cuda:0')
Nach 0 Schritten first: ist die rotation distanz zwischen den beiden Rotationen: tensor([2.0887], device='cuda:0')
Nach 0 Schritten second: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.8940], device='cuda:0')
Nach 1 Schritten first: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.7325], device='cuda:0')
Nach 1 Schritten second: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.5710], device='cuda:0')
Nach 2 Schritten first: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.4370], device='cuda:0')
Nach 2 Schritten second: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.3030], device='cuda:0')
Nach 3 Schritten first: ist die rotation distanz zwischen den beiden Rotationen: tensor([1.1919], device='cuda:0')
Nach 3 Schritten second: ist die rotation distanz zwischen den beiden Rotationen: tensor([1

## Schritte im Euclidean space

In [10]:
ziel = torch.randn(1, 10).cuda()
start = torch.zeros((1,10)).cuda()
schedule = LogUniformSchedule(165).return_schedule(128)
reverse_schedule = torch.flip(schedule, [0])

In [11]:
print(f"Am anfang ist die Euclidean distanz: {torch.linalg.norm(start - ziel)}")
# Compute the relative vector from start to ziel
relative_difference = ziel - start
step_size = 1
small_step = step_size * relative_difference
new_vector = small_step + start
print(f"Am Ende ist die Euclidean distanz: {torch.linalg.norm(new_vector - ziel)}")

Am anfang ist die Euclidean distanz: 2.464338541030884
Am Ende ist die Euclidean distanz: 0.0


## First order

In [12]:
print(f"Am anfang ist die Euclidean distanz: {torch.linalg.norm(start - ziel)}")
next = start.clone()
for i, (t_cur, t_next) in ts:
    # mache einen Schritt von start rotation in Richtung ziel rotation
    step_size = t_next - t_cur
    relative_difference = ziel - next
    small_step = - step_size * relative_difference / t_cur
    new_vector = small_step + next

    next = new_vector
    print(f"Nach {i} Schritten ist die Euclidean distanz: {torch.linalg.norm(next - ziel)}")
print(next, ziel)

Am anfang ist die Euclidean distanz: 2.464338541030884
Nach 0 Schritten ist die Euclidean distanz: 2.254178285598755
Nach 1 Schritten ist die Euclidean distanz: 2.0619401931762695
Nach 2 Schritten ist die Euclidean distanz: 1.8860965967178345
Nach 3 Schritten ist die Euclidean distanz: 1.7252473831176758
Nach 4 Schritten ist die Euclidean distanz: 1.5781170129776
Nach 5 Schritten ist die Euclidean distanz: 1.4435341358184814
Nach 6 Schritten ist die Euclidean distanz: 1.320428490638733
Nach 7 Schritten ist die Euclidean distanz: 1.2078214883804321
Nach 8 Schritten ist die Euclidean distanz: 1.1048165559768677
Nach 9 Schritten ist die Euclidean distanz: 1.0105971097946167
Nach 10 Schritten ist die Euclidean distanz: 0.9244125485420227
Nach 11 Schritten ist die Euclidean distanz: 0.8455771803855896
Nach 12 Schritten ist die Euclidean distanz: 0.7734657526016235
Nach 13 Schritten ist die Euclidean distanz: 0.7075040340423584
Nach 14 Schritten ist die Euclidean distanz: 0.6471676230430603


### Second order

In [13]:
print(f"Am anfang ist die Euclidean distanz: {torch.linalg.norm(start - ziel)}")
next = start.clone()
for i, (t_cur, t_next) in ts:
    # mache einen Schritt von start rotation in Richtung ziel rotation
    step_size = t_next - t_cur
    relative_difference = (ziel - next) / t_cur
    small_step = - step_size * relative_difference
    new_vector = small_step + next

    next = new_vector
    print(f"Nach {i} Schritten first ist die Euclidean distanz: {torch.linalg.norm(next - ziel)}")
    if i < len(ts) - 1:
        relative_difference_2nd = (ziel - next) / t_next
        small_step = - step_size * (relative_difference / 2 + relative_difference_2nd / 2)
        new_vector = small_step + next
        next = new_vector
        print(f"Nach {i} Schritten second ist die Euclidean distanz: {torch.linalg.norm(next - ziel)}")
print(next, ziel)

Am anfang ist die Euclidean distanz: 2.464338541030884
Nach 0 Schritten first ist die Euclidean distanz: 2.254178285598755
Nach 0 Schritten second ist die Euclidean distanz: 2.044017791748047
Nach 1 Schritten first ist die Euclidean distanz: 1.8697024583816528
Nach 1 Schritten second ist die Euclidean distanz: 1.6953868865966797
Nach 2 Schritten first ist die Euclidean distanz: 1.550803303718567
Nach 2 Schritten second ist die Euclidean distanz: 1.4062196016311646
Nach 3 Schritten first ist die Euclidean distanz: 1.2862950563430786
Nach 3 Schritten second ist die Euclidean distanz: 1.1663702726364136
Nach 4 Schritten first ist die Euclidean distanz: 1.066901445388794
Nach 4 Schritten second ist die Euclidean distanz: 0.9674326181411743
Nach 5 Schritten first ist die Euclidean distanz: 0.8849292397499084
Nach 5 Schritten second ist die Euclidean distanz: 0.8024259209632874
Nach 6 Schritten first ist die Euclidean distanz: 0.7339944839477539
Nach 6 Schritten second ist die Euclidean dist

## Wie kann man Schritte im Cholesky L Space machen?

In [14]:
scale = torch.randn(1, 3).cuda()
rotation = lietorch.SO3([], from_uniform_sampled=1).cuda().vec()
scale_unit = torch.ones(1, 3).cuda()
rotation_unit = lietorch.SO3(torch.tensor([[.0,.0,.0,1.]])).cuda().vec()
cov = build_covariance_from_scaling_rotation_xyzw(scale, rotation)
L = find_cholesky_L(cov)
cov_ziel = build_covariance_from_scaling_rotation_xyzw(scale_unit, rotation_unit)
L_ziel = find_cholesky_L(cov_ziel)

### Tangente Anlegen

In [15]:
print(f"Die geodesische distanz zwischen den beiden Cholesky Ls ist: {geodesic_distance(L, L_ziel)}")
next = L.clone()
for i, (t_cur, t_next) in ts:
    # mache einen Schritt von start rotation in Richtung ziel rotation
    step_size = t_next - t_cur
    # direction = log_map_at_L(L_ziel, next) / t_cur
    direction = log_map_at_L(K = next,L = L_ziel) / t_cur
    small_step = step_size * direction
    new_L = exp_map_at_L(X = small_step, L = next)

    next = new_L
    print(f"Nach {i} Schritten first ist die geodesische distanz: {geodesic_distance(next,L_ziel)}")
    if i < len(ts) - 1:
        # direction_2nd = log_map_at_L(L_ziel, next) / t_next
        direction_2nd = log_map_at_L(K= next,L = L_ziel) / t_next
        direction_combined = step_size * (direction_2nd / 2 + direction / 2)
        new_L = exp_map_at_L(X = direction_combined, L = next)
        # new_L = exp_map_at_L(small_step, next)
        next = new_L
        print(f"Nach {i} Schritten second ist die geodesische distanz: {geodesic_distance(next,L_ziel)}")
print(next, ziel)

Die geodesische distanz zwischen den beiden Cholesky Ls ist: tensor([1.1805], device='cuda:0')
Nach 0 Schritten first ist die geodesische distanz: tensor([0.9837], device='cuda:0')
Nach 0 Schritten second ist die geodesische distanz: tensor([0.8265], device='cuda:0')
Nach 1 Schritten first ist die geodesische distanz: tensor([0.7262], device='cuda:0')
Nach 1 Schritten second ist die geodesische distanz: tensor([0.6342], device='cuda:0')
Nach 2 Schritten first ist die geodesische distanz: tensor([0.5664], device='cuda:0')
Nach 2 Schritten second ist die geodesische distanz: tensor([0.5017], device='cuda:0')
Nach 3 Schritten first ist die geodesische distanz: tensor([0.4516], device='cuda:0')
Nach 3 Schritten second ist die geodesische distanz: tensor([0.4030], device='cuda:0')
Nach 4 Schritten first ist die geodesische distanz: tensor([0.3644], device='cuda:0')
Nach 4 Schritten second ist die geodesische distanz: tensor([0.3266], device='cuda:0')
Nach 5 Schritten first ist die geodesisc

### Relative L matrix berechnen
# manchmal laggy und führt nicht zum richtigen Ergebnis (numerische Gründe?)

In [16]:
print(f"Die geodesische distanz zwischen den beiden Cholesky Ls ist: {geodesic_distance(L, L_ziel)}")
next = L.clone()
for i, (t_cur, t_next) in ts:
    # mache einen Schritt von start rotation in Richtung ziel rotation
    step_size = t_next - t_cur
    group_op_res = group_operation_add(group_operation_inverse_L(next), L_ziel)
    # entweder so:
    # direction = log_map_at_L(group_op_res, next) / t_cur
    # small_step = - step_size * direction

    # oder so:
    direction = log_map_at_L(next, group_op_res) / t_cur
    small_step = step_size * direction
    new_L = exp_map_at_L(small_step, next)

    next = new_L
    print(f"Nach {i} Schritten first ist die geodesische distanz: {geodesic_distance(next,L_ziel)}")
    if i < len(ts) - 1:
        group_op_res = group_operation_add(group_operation_inverse_L(next), L_ziel)
        direction_2nd = log_map_at_L(group_op_res, next) / t_cur
        small_step = - step_size * (direction / 2 + direction_2nd / 2)
        new_L = exp_map_at_L(small_step, next)

        next = new_L
        print(f"Nach {i} Schritten second ist die geodesische distanz: {geodesic_distance(next,L_ziel)}")
print(next, L_ziel)

Die geodesische distanz zwischen den beiden Cholesky Ls ist: tensor([1.1805], device='cuda:0')
Nach 0 Schritten first ist die geodesische distanz: tensor([0.5700], device='cuda:0')
Nach 0 Schritten second ist die geodesische distanz: tensor([0.6123], device='cuda:0')
Nach 1 Schritten first ist die geodesische distanz: tensor([0.4835], device='cuda:0')
Nach 1 Schritten second ist die geodesische distanz: tensor([0.5042], device='cuda:0')
Nach 2 Schritten first ist die geodesische distanz: tensor([0.4045], device='cuda:0')
Nach 2 Schritten second ist die geodesische distanz: tensor([0.4187], device='cuda:0')
Nach 3 Schritten first ist die geodesische distanz: tensor([0.3391], device='cuda:0')
Nach 3 Schritten second ist die geodesische distanz: tensor([0.3494], device='cuda:0')
Nach 4 Schritten first ist die geodesische distanz: tensor([0.2846], device='cuda:0')
Nach 4 Schritten second ist die geodesische distanz: tensor([0.2924], device='cuda:0')
Nach 5 Schritten first ist die geodesisc